In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime
import hashlib

def make_remark_id(model_name, strategy):
    unique_string = f"{model_name}_{strategy}_{datetime.now()}"
    return hashlib.md5(unique_string.encode()).hexdigest()
    
conn = sqlite3.connect("../data/automated_metrics.db")
conn.execute("PRAGMA foreign_keys = ON;")
cursor = conn.cursor()

cases_df = pd.read_sql_query("SELECT * from hundred_sampled_transcript_and_context;", conn)
remark_addition_query = "INSERT INTO remark (remark_id, model, prompting_strategy, justice, " \
                        "remark_text, log_id, context_id) VALUES (?, ?, ?, ?, ?, ?, ?)"
additional_remarks = []
for index, row in cases_df.iterrows():
    generated_remark = row['turn_text']
    model_name = 'actual_text'
    log_id = 'manual'
    strategy = None
    remark_id = make_remark_id(model_name, strategy)
    additional_remarks.append((remark_id, model_name, strategy, row["justice"], generated_remark, 
                               log_id, row["context_id"]))

cursor.executemany(remark_addition_query, additional_remarks)
conn.commit()
cursor.close()
conn.close()